In [1]:
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from osgeo import gdal
from scipy.io import loadmat
from scipy.fft import ifft, fft, fftshift, ifftshift, fftfreq
from scipy.fft import ifft2, fft2
from scipy.constants import pi, c

In [129]:
PRIMARY_HEADER = [
    3,   # Packet Version Number
    1,   # Packet Type
    1,   # Secondary Header Flag
    7,   # Process ID
    4,   # Process Category
    2,   # Sequence Flags
    14,  # Packet Sequence Count 
    16   # Packet Data Length
]

SECONDARY_HEADER = [
    32,  # Coarse Time
    16,  # Fine Time
    32,  # Sync Marker
    32,  # Data Take ID
    8,   # ECC Number
    1,   # N/A
    3,   # Test Mode
    4,   # RX Channel ID
    32,  # Instrument Configuration ID
    8,   # Sub-Commutative Ancillary Data Word Index
    16,  # Sub-Commutative Ancillary Data Word
    32,  # Counter Service
    32,  # PRI Count
    1,   # Error Flag
    2,   # N/A
    5,   # BAQ Mode
    8,   # BAQ Block Length
    8,   # N/A
    8,   # Range Decimation
    8,   # RX Gain
    16,  # TX Ramp Rate
    16,  # Pulse Start Frequency
    24,  # Pulse Length
    3,   # N/A
    5,   # Rank
    24,  # PRI
    24,  # SWST
    24,  # SWL
    24,  # SAS SSB Message *** TODO: See Page 43, 44
    24,  # SES SSB Message *** TODO: See Page 50
    16,  # Number of Quadratures
    8,   # N/A
]

SUB_COMMUTATIVE_DATA_SERVICE = [
    16,      # Dummy Data
    64,      # X-Axis ECEF Position
    64,      # Y-Axis ECEF Position
    64,      # Z-Axis ECEF Position
    32,      # X-velocity ECEF
    32,      # Y-velocity ECEF
    32,      # Z-velocity ECEF
    16,      # POD Data Stamp 1
    16,      # Pod Data Stamp 2
    16,      # Pod Data Stamp 3
    16,      # Pod Data Stamp 4
    32,      # Q0 Attitude Quaternion
    32,      # Q1 Attitude Quaternion
    32,      # Q2 Attitude Quaternion
    32,      # Q3 Attitude Quaternion
    32,      # OmegaX Angular Rate
    32,      # OmegaY Angular Rate
    32,      # OmegaZ Angular Rate
    16,      # Data Time Stamp 1
    16,      # Data Time Stamp 2
    16,      # Data Time Stamp 3
    16,      # Data Time Stamp 4
    16,      # Pointing Status
    16,      # Temperature Update Status
    8, 8, 8, # Tile 1 EFE H, V Temperature and Activate TA Temperature
    8, 8, 8, # Tile 2
    8, 8, 8, # Tile 3
    8, 8, 8, # Tile 4
    8, 8, 8, # Tile 5
    8, 8, 8, # Tile 6
    8, 8, 8, # Tile 7
    8, 8, 8, # Tile 8
    8, 8, 8, # Tile 9
    8, 8, 8, # Tile 10
    8, 8, 8, # Tile 11
    8, 8, 8, # Tile 12
    8, 8, 8, # Tile 13
    8, 8, 8, # Tile 14
    9,       # N/A
    7,       # TGU Temperature
]

In [119]:
class Packet:
    def __init__(
        self,
        packet_version_number,
        packet_type,
        secondary_header_flag,
        process_id,
        process_category,
        sequence_flags,
        packet_sequence_count,
        packet_data_length,
        packet_secondary_header,
        user_data_field
    ):
        self.packet_version_number = packet_version_number
        self.packet_type = packet_type
        self.secondary_header_flag = secondary_header_flag
        self.process_id = process_id
        self.process_category = process_category,
        self.sequence_flags = sequence_flags
        self.packet_sequence_count = packet_sequence_count
        self.packet_data_length = packet_data_length
        
        self.packet_secondary_header = packet_secondary_header

        self.user_data_field = _decode_user_data_field()

    def _decode_user_data_field(self):
        pass

In [109]:
def read_and_pop(bit_string: str, bit_length: int):
    return bit_string[0:bit_length], bit_string[bit_length:]

def get_primary_header(primary_header_bytes):
    bit_string = ''
    for byte in primary_header_bytes:
        bit_string += '{:08b}'.format(byte)

    bit_lens = PRIMARY_HEADER
    header_value_array = []
    for bit_len in bit_lens:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    return {
        'packet_version_number': header_value_array[0],
        'packet_type': header_value_array[1],
        'secondary_header_flag': header_value_array[2],
        'process_id': header_value_array[3],
        'process_category': header_value_array[4],
        'sequence_flags': header_value_array[5],
        'packet_sequence_count': header_value_array[6],
        'packet_data_length': header_value_array[7],
    }


def get_secondary_header(secondary_header_bytes):
    bit_string = ''
    for byte in primary_header_bytes:
        bit_string += '{:08b}'.format(byte)

    bit_lens = SECONDARY_HEADER
    header_value_array = []
    for bit_len in bit_lens:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    return {
        'packet_version_number': header_value_array[0],
        'packet_type': header_value_array[1],
        'secondary_header_flag': header_value_array[2],
        'process_id': header_value_array[3],
        'process_category': header_value_array[4],
        'sequence_flags': header_value_array[5],
        'packet_sequence_count': header_value_array[6],
        'packet_data_length': header_value_array[7],
    }


def read_packets(l0_data_file):
    packets = []
    with open(l0_data_file, 'rb') as raw_data:
        primary_header_bytes = raw_data.read(6)
        primary_header = get_primary_header(primary_header_bytes)
        secondary_header_bytes = raw_data.read(62)
        secondary_header = get_secondary_header(secondary_header_bytes)

        # packet_version_number
        # packet_type
        # secondary_header_flag
        # process_id
        # process_category
        # sequence_flags
        # packet_sequence_count
        # packet_data_length
        # packet_secondary_header

        return header_field_dict

In [110]:
l0_data_file = 'sar_data/S1A_IW_RAW__0SDV_20240806T135224_20240806T135256_055093_06B68A_AE41.SAFE/s1a-iw-raw-s-vv-20240806t135224-20240806t135256-055093-06b68a.dat'

In [111]:
read_packets(l0_data_file)

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010101',
 'packet_data_length': '0100100010111101'}

In [116]:
bit_string = '0100100010111101'

In [130]:
sum(SECONDARY_HEADER) / 8

62.0